In [ ]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

%matplotlib inline

In [ ]:
def remove_jumps(df, col='tmpf', delta_lim=25):
    # Remove readings that jump more than 25F
    diff = abs(df[col].diff()) > delta_lim
    return df.mask(diff)
# Curve Fit
def objective(x, a, b, c, d, e, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + f
def curvefit(x,y):
    # curve fit
    popt, _ = curve_fit(objective, x, y)
    # summarize the parameter values
    a, b, c, d, e, f = popt
    # define a sequence of inputs between the smallest and largest known inputs
    x_line = np.arange(min(x), max(x), 1)
    # calculate the output for the range
    y_line = objective(x_line, a, b, c, d, e, f)
    return [x_line, y_line]

In [ ]:
df = pd.read_csv('SFO.csv')
df['datetime'] = pd.to_datetime(df['valid'])
df = df.set_index('datetime', drop=True)
df = remove_jumps(df)
df = df.dropna(how='any')

In [ ]:
df.plot()

In [ ]:
# Plot min daily temperature as quartiles
daily_stats = {}
df_min = df.resample('D').min()
for day, _df in df_min.groupby(df_min.index.day_of_year):
    daily_stats[day] = {
        'min':_df['tmpf'].min(),
        'max':_df['tmpf'].max(),
        'median':_df['tmpf'].median(),
        '25th':_df['tmpf'].quantile(0.25),
        '75th':_df['tmpf'].quantile(0.75)        
    }
daily_stats = pd.DataFrame(daily_stats).T

# Plot fitted median line
x,y = curvefit(daily_stats.index,daily_stats['median'])
fig, ax1 = plt.subplots(figsize=(10, 4))
ax1.plot(x,y,color='k',label='Median')

# Plot fitted 25th & 25th quantiles
x25,y25 = curvefit(daily_stats.index,daily_stats['25th'])
x75,y75 = curvefit(daily_stats.index,daily_stats['75th'])

ax1.fill_between(
    x25,
    y25,
    y75,
    facecolor='k',
    alpha=0.1,
    label='Q1-Q3'
)
ax1.plot(
    daily_stats['max'].head(365),
    color='r',
    linewidth=0.5,
    alpha=1,
    label='max'
)
ax1.plot(
    daily_stats['min'].head(365),
    color='b',
    linewidth=0.5,
    alpha=1,
    label='min'
)

lastyear = df_min[(df_min.index > (pd.to_datetime("2022-09-22")))
                  & (df_min.index < pd.to_datetime("2022-12-31"))
                 ]
ax1.plot(
    lastyear.index.day_of_year,
    lastyear['tmpf'],
    color='green'
)

thisyear = df_min[df_min.index > pd.to_datetime("2023-01-01")]
ax1.plot(
    thisyear.index.day_of_year,
    thisyear['tmpf'],
    color='green',
    label="Winter '22"
)
ax1.hlines(32, xmin=0, xmax=366, color='b')

plt.title('Historical Daily Low Temperature\n for San Francisco (SFO) 1980 - 2023')
plt.ylabel("Temperature [\N{DEGREE SIGN}F]")
plt.xticks(np.linspace(0,365,13)[:-1],
           ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec')
          )
plt.legend()
ax1.margins(x=0)
ax1.grid()